# Introduction to Graph-theory

[Graph-theory](https://en.wikipedia.org/wiki/Graph_theory) is the study of graphs,
which are mathematical structures used to model pairwise relations between objects.

Graphs are among the most ubiquitous models of both natural and human-made structures.
They can model many types of relations and process dynamics in physical, biological
and social systems. In computer science, they can represent networks of communication,
data organization, computational devices, the flow of computation, etc.
Graphs are one of the principal objects of study in
[discrete mathematics](https://en.wikipedia.org/wiki/Discrete_mathematics).

![6 nodes](images/6nodes.png)

| used name | description | alias | syntax |
|---|---|---|---|
| **node** | an intersection of edges | node, vertice, point | `g.add_node('x')` |
| **edge** | the line that connects intersections | edge, line, link | `g.add_edge('a','b')`|

A directed graph is a graph in which edges have orientations (e.g. edge node from,to node).
In graph-theory all edges have orientation. We declare this in Python using the following syntax:

In [1]:
from graph import Graph

g = Graph()
g.add_node('A')
g.add_node('B')
g.add_edge('A', 'B', value=4)

Technically seen it is not necessary to add the nodes first. You can just write
`g.add_edge('A', 'B')` and the Graph library will detect that the nodes haven't
been created and add them quietly.

To view the edges or nodes, use:

In [2]:
g.edges()

[('A', 'B', 4)]

In [3]:
g.nodes()

['A', 'B']

If you need to update the value on the edge just use `g.add_edge('A','B', value=10)` again.
The edge works like a dictionary, where `'A','B'` is the key and value `10` is the value.

In some cases it is not required for the graph to be directed. Mathematicians call this undirected.
In Python the mantra is *explicit is better than implicit*, so in graph-theory we omit
the value explicitly when adding edges and add it as bidirectional:  
`g.add_edge('A','B', bidirectional=True)`

If you need an undirected ***weighted graph*** you can add the value as weight:  
`g.add_edge('A','B', value=4, bidirectional=True)`

In other cases multiple edges between nodes are required. Graph-theory allows this using
dummy nodes. Here's an example:

In [4]:
from graph import Graph

g = Graph()
g.add_edge('A', 'B', value=4)  # a direct edge
g.add_edge('A', 'ab', value=2)  # dummy edge from A to dummy node ab
g.add_edge('ab', 'B', value=2)   # dummy edge from dummy node ab to B

Creating these dummy nodes by hand isn't very effective, so if you have a list of edges like this `L = [('A','B',4), ('A','B',3), ('A','B',2)]` it is good to know that nodes can be any hashable object, e.g. tuples, strings, numbers, ...

In [5]:
L = [('A','B',4), ('A','B',3), ('A','B',2)]
g = Graph()
for a,b,value in L:
    if g.edge(a,b) is None:
        g.add_edge(a,b,value)
    else:
        ab = a,b  # dummy node as tuple.
        g.add_edge(a, ab, value)
        g.add_edge(ab, b, value)

for edge in g.edges():
    print(edge)

('A', 'B', 4)
('A', ('A', 'B'), 2)
(('A', 'B'), 'B', 2)


The benefit of doing this is that most algorithms are simpler to implement than having to account
for multiple edges and dummy nodes are explicit. You will always know your list of nodes and can
hence exclude dummy nodes when reading through the list of nodes. For example, it would be silly
to search for a dummy node when looking for the shortest path. Rather, it is an obvious pragmatic
assumption that the algorithm will search for the shortest path between two meaningful points.

Since we are at this, you can count on all algorithms being available directly on the `Graph` class,
so that, for example the shortest path algorithm is available as:

```
g.shortest_path('A','B')
```

All the documentation is also available using the built-in help menu:

In [6]:
help(Graph.shortest_path)

Help on function shortest_path in module graph:

shortest_path(self, start, end, memoize=False)
    :param start: start node
    :param end: end node
    :param memoize: boolean (stores paths in a cache for faster repeated lookup)
    :return: distance, path as list



If you want a more detail description of what is going on, it is often helpful to directly to the specific function.

In [8]:
import inspect

In [10]:
print(inspect.getsource(Graph.shortest_path))

    def shortest_path(self, start, end, memoize=False):
        """
        :param start: start node
        :param end: end node
        :param memoize: boolean (stores paths in a cache for faster repeated lookup)
        :return: distance, path as list
        """
        if not memoize:
            return shortest_path(graph=self, start=start, end=end)

        if self._cache is None:
            self._cache = ShortestPathCache(graph=self)
        return self._cache.shortest_path(start, end)



Here you see that Graph wraps the function `shortest_path` and, should you choose to use the keyword `memoize=True` that it uses the class `ShortestPathCache`.

These can be inspected again in the same way:

In [11]:
from graph import shortest_path  # getting the function behind Graph.shortest_path
print(inspect.getsource(shortest_path))  # viewing the code

def shortest_path(graph, start, end):
    """ single source shortest path algorithm.
    :param graph: class Graph
    :param start: start node
    :param end: end node
    :return: distance, path (as list),
             returns float('inf'), [] if no path exists.
    """
    q, visited, minimums = [(0, 0, start, ())], set(), {start: 0}
    i = 1
    while q:
        (cost, _, v1, path) = heappop(q)
        if v1 not in visited:
            visited.add(v1)
            path = (v1, path)

            if v1 == end:  # exit criteria.
                L = []
                while path:
                    v, path = path[0], path[1]
                    L.append(v)
                L.reverse()
                return cost, L

            for _, v2, dist in graph.edges(from_node=v1):
                if v2 in visited:
                    continue
                prev = minimums.get(v2, None)
                next_node = cost + dist
                if prev is None or next_node < prev:
              

The code is very well annotated, so if the code doesn't explain itself, feel free to ask for a more elaborate example on the [github repo](https://github.com/root-11/graph-theory/issues).

Graph-theory tries to be transparent about everything it does. As the readme on the frontpage says: 

> with code you can explain to your boss